# Preparing the Data for Machine Learning
This notebook is used to prepare the data for machine learning by using Scikit Learn Pipelines to perform data imputation.

In [1]:
# For Deepnote to be able to use the custom libraries in the parent ../lib folder
import sys
sys.path.append('..')

In [ ]:
import numpy as np
import pandas as pd
from lib.read_data import read_and_join_output_file
from lib.impute import create_transformation_pipelines, group_train_test_split
from lib.viz import draw_missing_data_chart
from sklearn.decomposition import PCA
from sklearn import set_config

First we load and join all the datasets resulting from the ETL process and initialize some variables.

We have two potential targets for supervised and unsupervised learning, which we remove from the list of features:
* `GSE_GWE` - The Ground Surface Elevation to Groundwater Water Elevation - Depth to groundwater elevation in feet below ground surface
* `SHORTAGE_COUNT` -  The number of reported well shortages

In [ ]:
RANDOM_SEED = 42
X = read_and_join_output_file()

## Missing Data

In [ ]:
X.sample(5)

Let's look at the features with missing data.

In [ ]:
draw_missing_data_chart(X)

### Data Missing for Specific Years
Data were collected from the years 2014 to 2021 but some datasets only have data for specific years, when surveys were done/published. For example:
* Soils survey only has data for 2016
* Vegetations only has data for 2019
* Crops only has data for the the years 2014, 2016 and 2018
* Population density is available only for the years 2014 - 2020
* The reservoir water `PCT_OF_CAPACITY` is available only for the years 2018 - 2020

In [ ]:
crops_columns = [col for col in X if col.startswith('CROP_')]
crops_df = X[crops_columns].dropna()
soils_columns = [col for col in X if col.startswith('SOIL_')]
soils_df = X[soils_columns].dropna()
print(f"Years present in the Soils dataset {list(crops_df.index.unique(level='YEAR'))}")
print(f"Years present in the Crops dataset {list(soils_df.index.unique(level='YEAR'))}")

### Data Missing for Specific Township-Ranges
The Well Completion Reports dataset has data for all the years but have missing data for some specific Township-Ranges. Typically, if no wells were drilled in a specific Township-Range during the 2014-2020 period, then there is no data for that Township-Range for any of the following features:
* `TOTALDRILLDEPTH_AVG`
* `WELLYIELD_AVG`
* `STATICWATERLEVEL_AVG`
* `TOPOFPERFORATEDINTERVAL_AVG`
* `BOTTOMOFPERFORATEDINTERVAL_AVG`
* `GROUNDSURFACEELEVATION_AVG`
* `TOTALCOMPLETEDDEPTH_AVG`

Wells can also be reported with incomplete data, which means that some of the above features data could be missing for some Township-Ranges, even if wells were reported in those Township-Range.

In [ ]:
all_township_ranges = set(X.index.unique(level="TOWNSHIP_RANGE"))
wells_columns = [col for col in X if col.endswith('_AVG') or col == "TOWNSHIP_RANGE"]
wells_df = X[wells_columns].dropna()
missing_township_ranges = all_township_ranges - set(wells_df.index.unique(level="TOWNSHIP_RANGE"))
print(f"There are {len(missing_township_ranges)} out of {len(all_township_ranges)} Township-Ranges with missing well completion report data: {missing_township_ranges}")

## Train-Test Split
We split the dataset into a training and test set before doing data imputation. As we deal with time series data grouped at the Township-Range level, we can't split the dataset by randomly splitting rows between the train and test sets. We need to keep data of Township-Ranges together. To do that we use our custom train_test_split function which is based on scikit-learn `GroupShuffleSplit` class.

In [ ]:
X_train, X_test = group_train_test_split(X, RANDOM_SEED)

Let's look at 2 examples of the training and test sets.

In [ ]:
X_train.head(16)

In [ ]:
X_test.head(16)

## Data Imputation
### Imputation Strategies
To impute the missing data we will use the following strategies
1. We assume little year-to-year variation in Crops, Soils and Vegetation. The missing Crops data will thus be imputed from the previous year (e.g. the 2015 data will be set as the 2014 data). For the Soils and Vegetation where we only have data for 1 year, the missing data will all be imputed from the available year.
2. The 2021 population density data will be estimated based on the 2020 population density and the 2019-2020 trend.
3. For the pre-2018 missing reservoir water `PCT_OF_CAPACITY` data, as California was affected by sever droughts during those years, we will impute missing data by taking the **minimum** `PCT_OF_CAPACITY` for that Township-Range in the post 2018 data.
4. For the well completion reports' features with missing we will use 2 distinct strategies:
    * For the  `GROUNDSURFACEELEVATION_AVG` feature we will use the median values over all the years for that Township-Range. For Township-Ranges with no data at all for any of the 2014-2020 years, we will use the median value over all Township-Ranges.
    * For the other features they will be set to 0, since these are well measurements and missing data are mainly due to no wells being drilled in that Township-Range and year.

In [ ]:
impute_pipeline, columns = create_transformation_pipelines(X_train)
X_train_impute = impute_pipeline.fit_transform(X_train)
X_test_impute = impute_pipeline.transform(X_test)

We combine the imputed training and test datasets into one dataset to visualize the results.

In [ ]:
set_config(display="diagram")
display(impute_pipeline)

In [ ]:
X_train_impute_df = pd.DataFrame(X_train_impute, index=X_train.index, columns=columns)
X_test_impute_df = pd.DataFrame(X_test_impute, index=X_test.index, columns=columns)
X_impute_df = pd.concat([X_train_impute_df, X_test_impute_df], axis=0)
X_impute_df.sort_index(level=["TOWNSHIP_RANGE", "YEAR"], inplace=True)
X_impute_df.head(16)

In [ ]:
draw_missing_data_chart(X_impute_df)